# Goals and objectives of the project
Perform exploratory data analysis using the example of a dataset
with a target variable containing the values ​​of students' scores in mathematics

# Libraries import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sb
import statsmodels.api as sm
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from itertools import combinations

# Init

In [ ]:
sb.set_style("darkgrid")

# Functions

In [ ]:
def fillna_median(df, col):
    """
    Function fills NaN values in column of df with MEDIAN value.
    :param col: Column to replace NaN with median
    :param df: Dataframe with specified column
    """
    df.fillna({col : df[col].median()}, inplace=True)

def fillna_mode(df, col):
    """
    Function fills NaN values in column of df with MODE value.
    :param col: Column to replace NaN with mode
    :param df: Dataframe with specified column
    """
    df.fillna({col : df[col].mode().iloc[0]}, inplace=True)

def fill_IQR_outliers_median(df, col):
    """
    Function defines and fills outliers in column of df with MEDIAN value based on IQR criterion.
    :param col: Column to replace outliers with median
    :param df: Dataframe with specified column
    """
    q1, q3 = df[col].quantile([0.25, 0.75])
    IQR = q3 - q1
    df.loc[~df[col].between(q1-1.5*IQR, q3+1.5*IQR), col] = df[col].median()

def show_boxplot(df, qual_col, y_col):
    """
    Show boxplots for every qualitative value in specified column
    :param df: DataFrame
    :param qual_col: Column with qualitative values
    :param y_col: y variable for boxplot
    """
    n_unique = df[qual_col].nunique()
    fig, ax = plt.subplots(figsize = (1.5*n_unique, 4))
    sb.boxplot(x=qual_col, y=y_col, data=df, ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + qual_col)
    plt.show()

def check_ttest_diff(df, qual_col, dependent_var, p_value):
    """
    Compute Student's T-test statistics between all categories in col and print results
    :param df: DataFrame
    :param qual_col: Qualitative column with categories in interest in specified df
    :param dependent_var: Name of the column with dependent variable
    :param p_value: Desired p-value threshold
    :return: Test result for specified column:
                True - if T-test was passed
                       (there IS statistically significant differences within column)
                False - if T-test was failed
                        (there IS NO statistically significant differences within column)
    """
    categories = df[qual_col].unique()
    cat_combinations = list(combinations(categories, 2))
    n_cat = len(cat_combinations)
    for category_pair in cat_combinations:
        p = ttest_ind(df.loc[df[qual_col] == category_pair[0], dependent_var],
                      df.loc[df[qual_col] == category_pair[1], dependent_var]).pvalue
        # T-test with Bonferroni correction
        if p <= p_value/n_cat:
            print(f'Statistically significant differences via T-test were found for column \'{qual_col}\', p={p}')
            return True
        else:
            print(f'NO statistically significant differences via T-test were found for column \'{qual_col}\', p={p}')

    return False

def check_anova_diff(df, qual_col, dependent_var, p_value):
    """
    Compute Student's T-test statistics between all categories in col and print results
    :param df: DataFrame
    :param qual_col: Qualitative column with categories in interest in specified df
    :param dependent_var: Name of the column with dependent variable
    :param p_value: Desired p-value threshold
    :return: Test result for specified column:
                True - if T-test was passed
                       (there IS statistically significant differences within column)
                False - if T-test was failed
                        (there IS NO statistically significant differences within column)
    """
    categories = list(df[qual_col].unique())
    f, p = f_oneway(*[df.loc[df[qual_col] == cat, dependent_var] for cat in categories])
    if p<=p_value:
        result = True
        print(f'Statistically significant differences via ANOVA test were found for column \'{qual_col}\', F={f}, p={p}')
    else:
        result = False
        print(f'NO statistically significant differences via ANOVA test were found for column \'{qual_col}\', F={f}, p={p}')

    return result

# Data loading

In [ ]:
stud_df_orig = pd.read_csv('stud_math.xls')
stud_df_orig.sample(10)
stud_df_orig.info()
stud_df_orig.describe()

# Exploratory data analysis

In [ ]:
# In general our data rather "good" as we can judge on its text table analysis - no strange/corrupted values,
# so we can proceed directly with EDA

# Analyze missed values in data
missing_stat = {}
for col in stud_df_orig.columns:
    missing_stat[col] = round(100*stud_df_orig[col].isna().sum() / len(stud_df_orig.index), 2)
missing_stat = dict(sorted(missing_stat.items(), key=lambda item: item[1], reverse=True))
print('Missing value percents:')
display(missing_stat)

# Conclusion
# Max amount of missed values - 11.39%, so in general we have enough data for EDA

In [ ]:
# Fortunately our target variable "score" has comparably small amount of missing values (1,52%),
# because we need to drop this records for further analysis
stud_df = stud_df_orig.dropna(subset=['score'])
stud_df.info()
# now we have 389 rows in our stud_df DataFrame

In [ ]:
# Overview of the source data. Not very informative due to number of features, but still useful
sb.pairplot(stud_df)

## Primary data processing

In [ ]:
# Let's see on every column and preprocess them if necessary (NaN, outliers)

In [ ]:
# Column 'school'
stud_df['school'].value_counts(dropna=False)
# everything looks OK. We have two schools (GP and MS) and expectedly no NaN

In [ ]:
# Column 'sex'
stud_df['sex'].value_counts(dropna=False)
# again everything is OK.

In [ ]:
# Column 'age'
print(stud_df['age'].value_counts(dropna=False))
stud_df['age'].hist()
# everything is OK. No NaN values, no outliers (from description should be between 15 and 22 and it is).
# Wish all the data would be like this..;)

In [ ]:
# We know that in other columns with qualitative variables there are some NaN values
# Let's replace them with mode value in universal manner
# (including columns without NaN is OK, they just remain unchanged)
qualitative_cols = stud_df.select_dtypes(include='object').columns

for col in qualitative_cols:
    print('Processing column ', col.upper())
    print(stud_df[col].value_counts(dropna=False))

    fillna_mode(stud_df, col)

    print(stud_df[col].value_counts(dropna=False))

# Looking on the processing results we see no strange/corrupted nominal values
# and no NaN values after substitution with mode(), so far so good

In [ ]:
# Now process quantitative variables (excluding dependent variable 'score')
quantitative_cols = list(stud_df.select_dtypes(include=['int64', 'float64']).columns)
quantitative_cols.remove('score')

# df for straightforward outliers processing attempt
stud_df_outliers_test = stud_df.copy()

for col in quantitative_cols:
    print('Processing column ', col.upper())
    print('Number of NaN in ', col, ' = ', stud_df[col].isna().sum())

    fillna_median(stud_df, col)

    print('Number of NaN in ', col, ' after fillna = ', stud_df[col].isna().sum())

    # try process outliers in same loop
    # plot graphs before and after !very straightforward! dealing with outliers
    fig, axs = plt.subplots(2,2, figsize=[20,7.5], sharex='col')
    stud_df_outliers_test[col].hist(ax=axs[0,0])
    sb.boxplot(x=stud_df_outliers_test[col], ax=axs[0,1])

    fill_IQR_outliers_median(stud_df_outliers_test, col)

    stud_df_outliers_test[col].hist(ax=axs[1,0])
    sb.boxplot(x=stud_df_outliers_test[col], ax=axs[1,1])
    fig.suptitle(f'Plots for column {col.upper()} outliers elimination', fontsize=16)


# Conclusion
# Dealing with outliers of all columns in the same way via IQR threshold - not a good idea.
# Need common sense! So let's process outliers separately for each column

In [ ]:
# Column 'age'
print(stud_df['age'].value_counts(dropna=False))
# No outliers according to description (from 15 to 22)

In [ ]:
# Column 'Medu'
print(stud_df['Medu'].value_counts(dropna=False))
# No outliers according to description (from 0 to 4)

In [ ]:
# Column 'Fedu'
print(stud_df['Fedu'].value_counts(dropna=False))
# There is an obvious outlier: 40.
# Let's just guess that it should be 4.0 like others especially since 4.0 is nota mode but rather often value
stud_df.loc[stud_df['Fedu'] == 40,'Fedu'] = 4
print(stud_df['Fedu'].value_counts(dropna=False))

In [ ]:
# Column 'traveltime'
print(stud_df['traveltime'].value_counts(dropna=False))
# No outliers according to description (from 1 to 4)

In [ ]:
# Column 'studytime'
print(stud_df['studytime'].value_counts(dropna=False))
# No outliers according to description (from 1 to 4)

In [ ]:
# Column 'studytime, granular'
print(stud_df['studytime, granular'].value_counts(dropna=False))
# already can see the correlation in frequency with 'studytime'

In [ ]:
# Column 'failures'
print(stud_df['failures'].value_counts(dropna=False))
# No outliers according to description (from 0 to 3)

In [ ]:
# Column 'famrel'
print(stud_df['famrel'].value_counts(dropna=False))
# There is an obvious outlier: -1. Replace it with median
stud_df.loc[stud_df['famrel'] == -1,'famrel'] = stud_df['famrel'].median()
print(stud_df['famrel'].value_counts(dropna=False))

In [ ]:
# Column 'freetime'
print(stud_df['freetime'].value_counts(dropna=False))
# No outliers according to description (from 1 to 5)

In [ ]:
# Column 'goout'
print(stud_df['goout'].value_counts(dropna=False))
# No outliers according to description (from 1 to 5)

In [ ]:
# Column 'health'
print(stud_df['health'].value_counts(dropna=False))
# No outliers according to description (from 1 to 5)

In [ ]:
# Column 'absences'
print(stud_df['absences'].value_counts(dropna=False))
fig, axs = plt.subplots(3,1, figsize=[10,10])
sb.histplot(x=stud_df['absences'], ax=axs[0])
sb.boxplot(x=stud_df['absences'], ax=axs[1])
fig.suptitle(f'Plots for column absences outliers elimination', fontsize=16)

fill_IQR_outliers_median(stud_df, 'absences')

sb.histplot(x=stud_df['absences'], ax=axs[2])

Now all columns are ready for further analysis

In [ ]:
# One more global overview of the already preprocessed data. Looks better.
sb.pairplot(stud_df)

# Correlation analysis

In [ ]:
# calculate the correlation matrix
corr = stud_df.corr()
# plot the heatmap
sb.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

# Conclusions
# Variable 'studytime, granular' is strongly negatively correlated with 'studytime' and can be excluded.
# Father and mother education level is rather correlated with each other

In [ ]:
# So exclude 'studytime, granular' column from dataset
stud_df = stud_df.drop(columns='studytime, granular')
stud_df.info()

# Qualitative variables analysis

In [ ]:
for col in qualitative_cols:
    show_boxplot(stud_df, col, 'score')

# Conclusions
# From the generated plots we can see that there are several qualitative variables that have similar
# boxplots for our dependent variable score. So we should check their influence on score statistically.
# These variables are:
# ['sex', 'famsize', 'Pstatus', 'reason', 'famsup', 'paid', 'activity', 'nursury', 'romantic']
# Also maybe we should consider (in some real case) merging some of the similar categories in Mjob and Fjob in one category
# e.g. [other, health, services and at_home] for Fjob could be merged in one category (with additional stat analysis).
# One more observation: if someone's father is a !teacher! - this someone shows much better results in math 8^)

In [ ]:
# Considering the number of unique values in qualitative variables we will use Student's T-test for variables with only 2
# categories ['sex', 'famsize', 'Pstatus', 'famsup', 'paid', 'activities', 'nursery', 'romantic']
# and ANOVA (ANalysis Of VAriance) for feature 'reason' which consists of 4 categories with similar boxplots

In [ ]:
# In general before this we should also check if our variables have Normal distribution via such tests like
# Shapiro-Wilk Test, D’Agostino’s K^2 Test, Anderson-Darling Test and so on. But for this study case
# let's just check our data for "normality" via visual Q-Q plot

bin_qual_cols = ['sex', 'famsize', 'Pstatus', 'famsup', 'paid', 'activities', 'nursery', 'romantic']
mult_qual_cols = ['reason']
qual_cols_to_test = bin_qual_cols + mult_qual_cols

for qual_col in qual_cols_to_test:
    categories = stud_df[qual_col].unique()
    for category in categories:
        fig, ax = plt.subplots(1,1)
        category_sample = stud_df.loc[stud_df[qual_col] == category, 'score']
        sm.qqplot(category_sample.to_numpy(), ax=ax)
        fig.suptitle(f'Q-Q plot for qual_col {qual_col}, category {category}:', fontsize=16)
        plt.show()

# Conclusion for Q-Q plots
# Apart from the values of the initial quantiles we can say that we get rather "linish" plots, so
# make assumption that our data is normally distributed

In [ ]:
# Statistical test for binary qualitative variables
# We choose p value threshold = 0.15 (not 0.05) because we don't want to loose to much data for further processing,
# so with 1-0.15 = 85% confidence column categories have different mean (distribution?) and probably we NEED this data
# (tricky moment, need to be discussed with mentor)
non_informative_ttest_cols = []
for qual_col in bin_qual_cols:
    if not check_ttest_diff(stud_df, qual_col, 'score', 0.15):
        non_informative_ttest_cols.append(qual_col)

# Conclusion
# Statistically significant differences via T-test were found for some columns
# So from potentially non informative columns (bin_qual_cols) we will keep only them.

# Drop non informative columns
print('Non informative T-test columns are: ', non_informative_ttest_cols)
stud_df.drop(columns=non_informative_ttest_cols, inplace=True)
stud_df.info()

In [ ]:
# Statistical test for multicategorical qualitative variables
# We choose p value threshold = 0.2 (not 0.05) because we don't want to loose to much data for further processing,
# so with 1-0.15 = 85% confidence column categories have different mean (distribution?) and probably we NEED this data
# (tricky moment, need to be discussed with mentor)
non_informative_anova_cols = []
for qual_col in mult_qual_cols:
    if not check_anova_diff(stud_df, qual_col, 'score', 0.15):
        non_informative_anova_cols.append(qual_col)

# Conclusion
# NO statistically significant differences via ANOVA test were found for our single column 'reason', F=1.63, p=0.179
# So we can drop non informative columns (non_informative_anova_cols) from dataset

# Drop non informative columns
print('Non informative ANOVA columns are: ', non_informative_anova_cols)
stud_df.drop(columns=non_informative_anova_cols, inplace=True)
stud_df.info()

# Final conclusions

1. Our data have relatively small amount of missed values.
Max percentage of missed values among columns - 11.39% (Pstatus column)
2. Outliers were found only in 3 columns (for 2 of them there was only one outlier and for column 'absences'
 there were many of them), suggesting that the data are reasonably clean.
3. Additional column 'studytime, granular' was not informative due to perfect correlation with 'studytime'
4. Some qualitative columns were excluded due to relatively low statistical significance
of the 'score' values differences between their categories (but it's a matter of chosen p-value,
we chose it conservatively)
5. So the most important features for stud_math data finally are the following 21 columns:
['school','sex','age','address','Medu','Fedu','Mjob','Fjob','guardian','traveltime','studytime',
'failures','schoolsup','paid','higher','internet','romantic', 'famrel','freetime','goout','health',
'absences','score']